In [1]:
import os
from os import listdir
import pandas as pd
import io
from io import BytesIO

Pindel = 'Pindel/'
CNV = 'CNV/'

In [2]:
pindel_files = [f for f in listdir(Pindel)]
CNV_files = [f for f in listdir(CNV)]

In [3]:
len(pindel_files), len(CNV_files) #sorted the same way, no need to check if compared files are from the same individual

(64, 64)

In [4]:
def intersect(a, b):
    if len(a) > len(b): #change
        a, b = b, a

    intersection_list = [[0,0]]
    for i, el_a in enumerate(a):
        for j, el_b in enumerate(b):

            if el_a[0] <= el_b[0] <= el_b[1] <= el_a[1]: #inside (both b) ABBA
                added = False 

                for k, el_c in enumerate(intersection_list): #Check if such range already exist
                    if el_b[0] <= el_c[0] <= el_c[1] <= el_b[1]:
                        intersection_list[k] = [el_b[0], el_b[1]] #entend existing record
                        added = True
                    elif el_c[0] <= el_b[0] <= el_b[1] <= el_c[1]:
                        added = True
                        pass #it already exist
                    elif el_c[0] <= el_b[0] <= el_c[1] <= el_b[1]:
                        added = True
                        intersection_list[k] = [el_c[0], el_b[1]] #entend existing record
                    elif el_b[0] <= el_c[0] <= el_b[1] <= el_c[1]:
                        added = True
                        intersection_list[k] = [el_b[0], el_c[1]] #entend existing record
                    else:
                        added = False
                if added == False:
                        intersection_list.append([el_b[0], el_b[1]])
                        
            elif el_a[0] <= el_b[0] <= el_a[1] <= el_b[1]: #cut from right ABAB
                added = False 
       
                for k, el_c in enumerate(intersection_list):
                    if el_b[0] <= el_c[0] <= el_c[1] <= el_a[1]:
                        added = True 
                        intersection_list[k] = [el_b[0], el_a[1]] #entend existing record
                    elif el_c[0] <= el_b[0] <= el_a[1] <= el_c[1]:
                        added = True 
                        pass #it already exist
                    elif el_c[0] <= el_b[0] <= el_c[1] <= el_a[1]:
                        added = True 
                        intersection_list[k] = [el_c[0], el_a[1]] #entend existing record
                    elif el_b[0] <= el_c[0] <= el_a[1] <= el_c[1]:
                        added = True 
                        intersection_list[k] = [el_b[0], el_c[1]] #entend existing record
                    else:
                        added = False
                if added == False:
                        intersection_list.append([el_b[0], el_a[1]])
                        
            elif el_b[0] <= el_a[0] <= el_a[1] <= el_b[1]: #both BAAB
                added = False 
 
                for k, el_c in enumerate(intersection_list):
                    if el_a[0] <= el_c[0] <= el_c[1] <= el_a[1]:
                        added = True 
                        intersection_list[k] = [el_a[0], el_a[1]] #entend existing record
                    elif el_c[0] <= el_a[0] <= el_a[1] <= el_c[1]:
                        added = True 
                        pass #it already exist
                    elif el_c[0] <= el_a[0] <= el_c[1] <= el_a[1]:
                        added = True 
                        intersection_list[k] = [el_c[0], el_a[1]] #entend existing record
                    elif el_a[0] <= el_c[0] <= el_a[1] <= el_c[1]:
                        added = True 
                        intersection_list[k] = [el_a[0], el_c[1]] #entend existing record
                    else:
                        added = False
                if added == False:
                        intersection_list.append([el_a[0], el_a[1]])

                        
            elif el_b[0] <= el_a[0] <= el_b[1] <= el_a[1]: #BABA
                added = False 
                
                for k, el_c in enumerate(intersection_list):
                    if el_a[0] <= el_c[0] <= el_c[1] <= el_b[1]:
                        intersection_list[k] = [el_b[0], el_a[1]] #entend existing record
                    elif el_c[0] <= el_a[0] <= el_b[1] <= el_c[1]:
                        pass #it already exist
                    elif el_c[0] <= el_a[0] <= el_c[1] <= el_b[1]:
                        intersection_list[k] = [el_c[0], el_a[1]] #entend existing record
                    elif el_a[0] <= el_c[0] <= el_b[1] <= el_c[1]:
                        intersection_list[k] = [el_a[0], el_c[1]] #entend existing record
                    else:
                        added = False
                if added == False:
                        intersection_list.append([el_a[0], el_b[1]])
            else:
                pass
    return intersection_list

In [9]:
for i in range(0,64): #0,64 by files
    file_pindel = pd.read_csv("Pindel/"+pindel_files[i], names=['Chr', "Start", "End"],  sep=" ", dtype={'Chr':'str', 'Start':'int', 'End':'int' })
    file_cnv = pd.read_csv("CNV/"+CNV_files[i], names=['Chr', "Start", "End"],  sep=" ", dtype={'Chr':'str', 'Start':'int', 'End':'int' })                            
    df_connected = pd.DataFrame(columns=['Chr', "Start", "End"])

    for j in range(1,30+1): #by chromosoms 1,30+1
        if j == 30:
            j = str("X")
        else:
            j = str(j)
        inter1 =  file_pindel[['Start', 'End']][file_pindel.Chr==j].values.tolist()
        inter2 =  file_cnv[['Start', 'End']][file_cnv.Chr==j].values.tolist()
        intersection_list = intersect(inter1, inter2)
        print("File:", i+1, " Intersection:", len(intersection_list), "Chromosom:", j)

        df_new = pd.DataFrame(intersection_list, columns =['Start', 'End'])[1:] 
        
        df_new.insert(0, "Chr", [j for i in range(len(intersection_list[1:]))])
        df_connected = pd.concat([df_connected,df_new])
        
    #Save for this individual
    df_connected.to_csv("Connected/"+str(pindel_files[i][:-13]) +" " + str(i+1) + ".csv", index=False)
    print("Saving")
    

File: 1 , Intersection: 1439 Chromosom: 1
File: 1 , Intersection: 1098 Chromosom: 2
File: 1 , Intersection: 1119 Chromosom: 3
File: 1 , Intersection: 1158 Chromosom: 4
File: 1 , Intersection: 1327 Chromosom: 5
File: 1 , Intersection: 1047 Chromosom: 6
File: 1 , Intersection: 1144 Chromosom: 7
File: 1 , Intersection: 936 Chromosom: 8
File: 1 , Intersection: 789 Chromosom: 9
File: 1 , Intersection: 548 Chromosom: 10
File: 1 , Intersection: 886 Chromosom: 11
File: 1 , Intersection: 890 Chromosom: 12
File: 1 , Intersection: 522 Chromosom: 13
File: 1 , Intersection: 621 Chromosom: 14
File: 1 , Intersection: 908 Chromosom: 15
File: 1 , Intersection: 902 Chromosom: 16
File: 1 , Intersection: 671 Chromosom: 17
File: 1 , Intersection: 1048 Chromosom: 18
File: 1 , Intersection: 1063 Chromosom: 19
File: 1 , Intersection: 437 Chromosom: 20
File: 1 , Intersection: 916 Chromosom: 21
File: 1 , Intersection: 458 Chromosom: 22
File: 1 , Intersection: 547 Chromosom: 23
File: 1 , Intersection: 283 Chromo

File: 7 , Intersection: 1019 Chromosom: 17
File: 7 , Intersection: 1975 Chromosom: 18
File: 7 , Intersection: 1996 Chromosom: 19
File: 7 , Intersection: 494 Chromosom: 20
File: 7 , Intersection: 1003 Chromosom: 21
File: 7 , Intersection: 1034 Chromosom: 22
File: 7 , Intersection: 934 Chromosom: 23
File: 7 , Intersection: 273 Chromosom: 24
File: 7 , Intersection: 1312 Chromosom: 25
File: 7 , Intersection: 632 Chromosom: 26
File: 7 , Intersection: 343 Chromosom: 27
File: 7 , Intersection: 317 Chromosom: 28
File: 7 , Intersection: 1116 Chromosom: 29
File: 7 , Intersection: 1669 Chromosom: X
Saving
File: 8 , Intersection: 547 Chromosom: 1
File: 8 , Intersection: 76 Chromosom: 2
File: 8 , Intersection: 98 Chromosom: 3
File: 8 , Intersection: 247 Chromosom: 4
File: 8 , Intersection: 217 Chromosom: 5
File: 8 , Intersection: 122 Chromosom: 6
File: 8 , Intersection: 220 Chromosom: 7
File: 8 , Intersection: 329 Chromosom: 8
File: 8 , Intersection: 53 Chromosom: 9
File: 8 , Intersection: 59 Chrom

File: 14 , Intersection: 1369 Chromosom: 1
File: 14 , Intersection: 89 Chromosom: 2
File: 14 , Intersection: 327 Chromosom: 3
File: 14 , Intersection: 427 Chromosom: 4
File: 14 , Intersection: 584 Chromosom: 5
File: 14 , Intersection: 294 Chromosom: 6
File: 14 , Intersection: 349 Chromosom: 7
File: 14 , Intersection: 813 Chromosom: 8
File: 14 , Intersection: 269 Chromosom: 9
File: 14 , Intersection: 65 Chromosom: 10
File: 14 , Intersection: 193 Chromosom: 11
File: 14 , Intersection: 181 Chromosom: 12
File: 14 , Intersection: 138 Chromosom: 13
File: 14 , Intersection: 363 Chromosom: 14
File: 14 , Intersection: 195 Chromosom: 15
File: 14 , Intersection: 136 Chromosom: 16
File: 14 , Intersection: 51 Chromosom: 17
File: 14 , Intersection: 132 Chromosom: 18
File: 14 , Intersection: 29 Chromosom: 19
File: 14 , Intersection: 21 Chromosom: 20
File: 14 , Intersection: 219 Chromosom: 21
File: 14 , Intersection: 30 Chromosom: 22
File: 14 , Intersection: 61 Chromosom: 23
File: 14 , Intersection: 2

File: 20 , Intersection: 193 Chromosom: 12
File: 20 , Intersection: 358 Chromosom: 13
File: 20 , Intersection: 370 Chromosom: 14
File: 20 , Intersection: 47 Chromosom: 15
File: 20 , Intersection: 157 Chromosom: 16
File: 20 , Intersection: 36 Chromosom: 17
File: 20 , Intersection: 133 Chromosom: 18
File: 20 , Intersection: 45 Chromosom: 19
File: 20 , Intersection: 3 Chromosom: 20
File: 20 , Intersection: 93 Chromosom: 21
File: 20 , Intersection: 13 Chromosom: 22
File: 20 , Intersection: 47 Chromosom: 23
File: 20 , Intersection: 23 Chromosom: 24
File: 20 , Intersection: 19 Chromosom: 25
File: 20 , Intersection: 14 Chromosom: 26
File: 20 , Intersection: 750 Chromosom: 27
File: 20 , Intersection: 39 Chromosom: 28
File: 20 , Intersection: 444 Chromosom: 29
File: 20 , Intersection: 932 Chromosom: X
Saving
File: 21 , Intersection: 1124 Chromosom: 1
File: 21 , Intersection: 1369 Chromosom: 2
File: 21 , Intersection: 1419 Chromosom: 3
File: 21 , Intersection: 1103 Chromosom: 4
File: 21 , Inters

File: 26 , Intersection: 270 Chromosom: 27
File: 26 , Intersection: 16 Chromosom: 28
File: 26 , Intersection: 291 Chromosom: 29
File: 26 , Intersection: 820 Chromosom: X
Saving
File: 27 , Intersection: 1314 Chromosom: 1
File: 27 , Intersection: 856 Chromosom: 2
File: 27 , Intersection: 1083 Chromosom: 3
File: 27 , Intersection: 1116 Chromosom: 4
File: 27 , Intersection: 1591 Chromosom: 5
File: 27 , Intersection: 1148 Chromosom: 6
File: 27 , Intersection: 1270 Chromosom: 7
File: 27 , Intersection: 864 Chromosom: 8
File: 27 , Intersection: 803 Chromosom: 9
File: 27 , Intersection: 481 Chromosom: 10
File: 27 , Intersection: 1196 Chromosom: 11
File: 27 , Intersection: 935 Chromosom: 12
File: 27 , Intersection: 596 Chromosom: 13
File: 27 , Intersection: 635 Chromosom: 14
File: 27 , Intersection: 823 Chromosom: 15
File: 27 , Intersection: 873 Chromosom: 16
File: 27 , Intersection: 916 Chromosom: 17
File: 27 , Intersection: 1309 Chromosom: 18
File: 27 , Intersection: 1017 Chromosom: 19
File: 

File: 33 , Intersection: 848 Chromosom: 9
File: 33 , Intersection: 698 Chromosom: 10
File: 33 , Intersection: 1682 Chromosom: 11
File: 33 , Intersection: 830 Chromosom: 12
File: 33 , Intersection: 707 Chromosom: 13
File: 33 , Intersection: 750 Chromosom: 14
File: 33 , Intersection: 1090 Chromosom: 15
File: 33 , Intersection: 1116 Chromosom: 16
File: 33 , Intersection: 1140 Chromosom: 17
File: 33 , Intersection: 2160 Chromosom: 18
File: 33 , Intersection: 2065 Chromosom: 19
File: 33 , Intersection: 466 Chromosom: 20
File: 33 , Intersection: 1486 Chromosom: 21
File: 33 , Intersection: 1102 Chromosom: 22
File: 33 , Intersection: 848 Chromosom: 23
File: 33 , Intersection: 327 Chromosom: 24
File: 33 , Intersection: 1308 Chromosom: 25
File: 33 , Intersection: 627 Chromosom: 26
File: 33 , Intersection: 347 Chromosom: 27
File: 33 , Intersection: 369 Chromosom: 28
File: 33 , Intersection: 1302 Chromosom: 29
File: 33 , Intersection: 1466 Chromosom: X
Saving
File: 34 , Intersection: 652 Chromosom

File: 39 , Intersection: 995 Chromosom: 21
File: 39 , Intersection: 545 Chromosom: 22
File: 39 , Intersection: 627 Chromosom: 23
File: 39 , Intersection: 261 Chromosom: 24
File: 39 , Intersection: 956 Chromosom: 25
File: 39 , Intersection: 535 Chromosom: 26
File: 39 , Intersection: 242 Chromosom: 27
File: 39 , Intersection: 216 Chromosom: 28
File: 39 , Intersection: 1025 Chromosom: 29
File: 39 , Intersection: 1596 Chromosom: X
Saving
File: 40 , Intersection: 827 Chromosom: 1
File: 40 , Intersection: 63 Chromosom: 2
File: 40 , Intersection: 129 Chromosom: 3
File: 40 , Intersection: 287 Chromosom: 4
File: 40 , Intersection: 305 Chromosom: 5
File: 40 , Intersection: 178 Chromosom: 6
File: 40 , Intersection: 278 Chromosom: 7
File: 40 , Intersection: 407 Chromosom: 8
File: 40 , Intersection: 72 Chromosom: 9
File: 40 , Intersection: 53 Chromosom: 10
File: 40 , Intersection: 56 Chromosom: 11
File: 40 , Intersection: 303 Chromosom: 12
File: 40 , Intersection: 109 Chromosom: 13
File: 40 , Inter

File: 46 , Intersection: 111 Chromosom: 4
File: 46 , Intersection: 119 Chromosom: 5
File: 46 , Intersection: 176 Chromosom: 6
File: 46 , Intersection: 274 Chromosom: 7
File: 46 , Intersection: 240 Chromosom: 8
File: 46 , Intersection: 23 Chromosom: 9
File: 46 , Intersection: 36 Chromosom: 10
File: 46 , Intersection: 40 Chromosom: 11
File: 46 , Intersection: 271 Chromosom: 12
File: 46 , Intersection: 210 Chromosom: 13
File: 46 , Intersection: 257 Chromosom: 14
File: 46 , Intersection: 34 Chromosom: 15
File: 46 , Intersection: 107 Chromosom: 16
File: 46 , Intersection: 29 Chromosom: 17
File: 46 , Intersection: 79 Chromosom: 18
File: 46 , Intersection: 36 Chromosom: 19
File: 46 , Intersection: 9 Chromosom: 20
File: 46 , Intersection: 72 Chromosom: 21
File: 46 , Intersection: 8 Chromosom: 22
File: 46 , Intersection: 62 Chromosom: 23
File: 46 , Intersection: 17 Chromosom: 24
File: 46 , Intersection: 16 Chromosom: 25
File: 46 , Intersection: 13 Chromosom: 26
File: 46 , Intersection: 459 Chro

File: 52 , Intersection: 199 Chromosom: 16
File: 52 , Intersection: 16 Chromosom: 17
File: 52 , Intersection: 145 Chromosom: 18
File: 52 , Intersection: 28 Chromosom: 19
File: 52 , Intersection: 25 Chromosom: 20
File: 52 , Intersection: 114 Chromosom: 21
File: 52 , Intersection: 44 Chromosom: 22
File: 52 , Intersection: 63 Chromosom: 23
File: 52 , Intersection: 43 Chromosom: 24
File: 52 , Intersection: 76 Chromosom: 25
File: 52 , Intersection: 17 Chromosom: 26
File: 52 , Intersection: 507 Chromosom: 27
File: 52 , Intersection: 86 Chromosom: 28
File: 52 , Intersection: 787 Chromosom: 29
File: 52 , Intersection: 917 Chromosom: X
Saving
File: 53 , Intersection: 169 Chromosom: 1
File: 53 , Intersection: 214 Chromosom: 2
File: 53 , Intersection: 275 Chromosom: 3
File: 53 , Intersection: 251 Chromosom: 4
File: 53 , Intersection: 380 Chromosom: 5
File: 53 , Intersection: 248 Chromosom: 6
File: 53 , Intersection: 298 Chromosom: 7
File: 53 , Intersection: 233 Chromosom: 8
File: 53 , Intersectio

File: 58 , Intersection: 386 Chromosom: 29
File: 58 , Intersection: 921 Chromosom: X
Saving
File: 59 , Intersection: 1255 Chromosom: 1
File: 59 , Intersection: 1007 Chromosom: 2
File: 59 , Intersection: 828 Chromosom: 3
File: 59 , Intersection: 966 Chromosom: 4
File: 59 , Intersection: 1510 Chromosom: 5
File: 59 , Intersection: 1094 Chromosom: 6
File: 59 , Intersection: 1314 Chromosom: 7
File: 59 , Intersection: 888 Chromosom: 8
File: 59 , Intersection: 826 Chromosom: 9
File: 59 , Intersection: 563 Chromosom: 10
File: 59 , Intersection: 1183 Chromosom: 11
File: 59 , Intersection: 1106 Chromosom: 12
File: 59 , Intersection: 639 Chromosom: 13
File: 59 , Intersection: 708 Chromosom: 14
File: 59 , Intersection: 995 Chromosom: 15
File: 59 , Intersection: 943 Chromosom: 16
File: 59 , Intersection: 944 Chromosom: 17
File: 59 , Intersection: 1242 Chromosom: 18
File: 59 , Intersection: 1203 Chromosom: 19
File: 59 , Intersection: 369 Chromosom: 20
File: 59 , Intersection: 1006 Chromosom: 21
File